In [2]:
import os
from pathlib import Path

try:
    from dotenv import find_dotenv, load_dotenv  
except ImportError as e:
    raise RuntimeError(
        "python-dotenv is not installed. Install it with: pip install python-dotenv"
    ) from e

env_path = find_dotenv(filename=".env", usecwd=True)
if env_path:
    load_dotenv(env_path)
    print(f"✅ Loaded .env: {env_path}")
else:
    # Still attempt default behavior (may load from other locations)
    load_dotenv()
    print(f"No .env found via find_dotenv(). CWD: {Path.cwd()}")

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise RuntimeError(
        "Missing GOOGLE_API_KEY. Ensure your .env is in the current working directory "
        f"({Path.cwd()}) and contains GOOGLE_API_KEY=... then restart the kernel and re-run this cell."
    )

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
if not TAVILY_API_KEY:
    raise RuntimeError(
        "Missing TAVILY_API_KEY. Ensure your .env is in the current working directory "
        f"({Path.cwd()}) and contains TAVILY_API_KEY=... then restart the kernel and re-run this cell."
    )

os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise RuntimeError(
        "Missing GROQ_API_KEY. Ensure your .env is in the current working directory "
        f"({Path.cwd()}) and contains GROQ_API_KEY=... then restart the kernel and re-run this cell."
    )

os.environ["GROQ_API_KEY"] = GROQ_API_KEY

print("✅ Setup and authentication complete.")


✅ Loaded .env: c:\Kaggle-agent\.env
✅ Setup and authentication complete.


In [3]:
from typing import Any, Dict
import json
import requests
import subprocess
import time
import uuid
import warnings

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.adk.agents import ParallelAgent, SequentialAgent
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)
from google.adk.a2a.utils.agent_to_a2a import to_a2a

warnings.filterwarnings("ignore")

print(" ADK components imported successfully.")

 ADK components imported successfully.


In [4]:
# HELPER FUNCTION

async def run_session(runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [5]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [6]:
import litellm
from google.adk.models.lite_llm import LiteLlm
from google.adk.agents import LlmAgent

litellm.set_verbose = False
litellm.suppress_debug_info = True

# Setup Services and model
memory_service = InMemoryMemoryService()
session_service = InMemorySessionService()

# llm_model = Gemini(model_id="gemini-2.0-flash-exp", retry_options=retry_config)

# llm_model = LiteLlm(
#     model="openrouter/meta-llama/llama-3.3-70b-instruct:free",
#     api_key=os.getenv("OPENROUTER_API_KEY"),
#     api_base="https://openrouter.ai/api/v1",
# )

# llm_model = LiteLlm(
#     model="ollama_chat/qwen2.5:1.5b",
#     api_base="http://localhost:11434",
# )

llm_model = LiteLlm(
    model="openai/llama-3.1-8b-instant", 
    # model="groq/compound",
    # model="openai/gpt-oss-120b",
    # model="openai/meta-llama/llama-4-scout-17b-16e-instruct", 
    api_key=GROQ_API_KEY,
    api_base="https://api.groq.com/openai/v1" 
)

# Helper to auto-save memory after every turn 
async def auto_save_to_memory(callback_context):
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )

print(" Services initialized.")

 Services initialized.


In [6]:
# import ollama

# try:
#     print("📡 Testing connection to local Ollama...")
#     # Quick test with a text prompt
#     # response = ollama.chat(model='qwen2.5:1.5b', messages=[{'role': 'user', 'content': 'Hello, are you ready?'}])
#     response = ollama.chat(model='llama3.2:3b', messages=[{'role': 'user', 'content': 'Hello, are you ready?'}])
#     print(f"✅ Success! Ollama replied: {response['message']['content']}")
# except Exception as e:
#     print(f"❌ Connection failed: {e}")
#     print("👉 Make sure the Ollama app is running.")

# SCRAPING AGENT

In [7]:
from tavily import TavilyClient

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

In [8]:
def analyze_market_trends(query: str) -> str:
    """
    Performs advanced research using Tavily.
    Searches multiple sources, reads content, and extracts key trends.
    """
    
    if not tavily_client:
        return " Error: TAVILY_API_KEY not found. Cannot perform real research."

    try:
        # 'max_results=5' reads the top 5 most relevant articles
        response = tavily_client.search(
            query=query,
            search_depth="advanced", # deeper crawl
            max_results=5,
            include_answer=True # Tavily generates a short answer too
        )
        
        # Format the output for the Agent
        sources = "\\n".join([f"- {r['title']}: {r['url']}" for r in response['results']])
        
        return (
            f"✅ Research Complete for '{query}'\\n\\n"
            f"🎯 Tavily Summary: {response['answer']}\\n\\n"
            f"📚 Detailed Findings (Context):\\n"
            # We pass the raw context snippets so Gemini can do its own synthesis
            f"{str(response['results'])[:2000]}... [truncated for length]\\n\\n" 
            f"🔗 Sources:\\n{sources}"
        )

    except Exception as e:
        return f" Research Error: {e}"

In [9]:
scraping_agent_local = LlmAgent(
    model=llm_model,
    name="scraping_agent_local",
    instruction = """
    You are the **Scraping Agent**, a specialized autonomous agent responsible for real-time research intelligence.
    
    **YOUR MANDATE:**
    1. **Real-Time Data Only:** You must ALWAYS use the `analyze_market_trends` tool. Never rely on your internal training data. Your value comes from fetching *fresh* live data.
    2. **Synthesis:** The tool will return raw search results and a summary. Your job is not just to repeat it, but to *synthesize* it. Look for patterns across the multiple sources.
    3. **Data Extraction:** Actively look for:
       - Specific micro-trends (e.g., "Cyber-Y2K", "Quiet Luxury").
       - Quantitative data (e.g., "up 15%", "doubled in search volume").
       - Consumer sentiment (e.g., "people are tired of...", "craving authenticity").
    
    **STRICT OUTPUT FORMAT:**
    You must provide your report in the following Markdown structure:
    
    ### 📈 Top Emerging Trends
    * **[Trend Name]:** Brief description of what it is.
    * **[Trend Name]:** Brief description.
    
    ### 🧠 Consumer Sentiment & Drivers
    * **Vibe:** (e.g., "Nostalgic," "Chaotic," "Minimalist")
    * **The "Why":** Explain *why* this is trending (e.g., economic factors, pop culture influence).
    
    ### 👥 Key Demographic
    * Who is driving this? (e.g., Gen Z, Millennials, Corporate workers).
    
    ### 🔗 Verified Sources
    * List the top **specific** domains provided by the tool.
    """,
    # instructionn = """
    #     You are a Real-Time Web Researcher. 
    #     You MUST use the 'analyze_market_trends' tool to fetch live data. Do not rely on your internal knowledge.

    #     Output a strict Markdown report with three sections:
    #     1. **Live Summary**: A 2-sentence overview of the absolute latest trends.
    #     2. **Key Data Points**: 3 bullet points with concrete metrics, dates, or quotes.
    #     3. **Sources**: A bulleted list of the exact URLs used.
    # """,
    tools=[analyze_market_trends]
    # tools=[google_search]
)

In [10]:
APP_NAME = "AutoMemoryApp"
USER_ID = "demo_user"

runner = Runner(
    agent=scraping_agent_local,
    app_name=APP_NAME,
    session_service=session_service,  
    memory_service=memory_service,    
)

await run_session(
    runner,
    "What are the latest AI trends?",
    "demo-research"
)


### Session: demo-research

User > What are the latest AI trends?
Model: > ### 📈 Top Emerging Trends
* **Explainable AI (XAI):** Efforts to make AI more transparent and understandable to humans.
* **Quantum AI:** The integration of quantum computing and artificial intelligence.
* **Generative AI:** The development of AI models that can generate new content, such as images, music, and text.

### 🧠 Consumer Sentiment & Drivers
* **Vibe:** Excitement, curiosity
* **The "Why":** The increasing demand for more human-like interactions and the need for businesses to stay competitive in a rapidly changing market.

### 👥 Key Demographic
* Gen Z and tech-savvy individuals who are driving the adoption of new AI technologies.

### 🔗 Verified Sources
* https://www.researchgate.net
* https://arxiv.org
* https://papers.ssrn.com


In [ ]:
def analyze_market_trends(query: str) -> str:
    """Performs market research using Tavily."""
    try:
        response = tavily_client.search(
            query=query,
            search_depth="advanced",
            max_results=5,
            include_answer=True
        )
        sources = "\n".join([f"- {r['title']}: {r['url']}" for r in response['results']])
        return f"SUMMARY: {response['answer']}\n\nDETAILS: {str(response['results'])[:1000]}\n\nSOURCES:\n{sources}"
    except Exception as e:
        return f"Error: {e}"



scraping_agent_local = LlmAgent(
    model=llm_model,
    name="scraping_agent_shard",
    instruction="""
    You are the Scraping Agent.
    ALWAYS call 'analyze_market_trends' to find live data.


    Rules:
    1) Make 1-3 targeted queries.
    2) After the tool returns, produce ONE final structured Markdown report.
    3) Do not output partial tool-call text, placeholders, or phrases like "pending".
    4) ONLY use information from the tool output. Do NOT infer from any video content or any other agent output.
    5) Do not claim "AI"/"machine learning" unless the tool output explicitly supports it.

    Output format (Markdown):
    - Trend Summary (3-6 bullets)
    - Key Evidence (2-5 bullets with concrete facts)
    - Sources (bulleted list of URLs strictly copied from the tool output; if none are present, write "Sources: None")
    """,
    tools=[analyze_market_trends]
)

In [10]:
APP_NAME = "AutoMemoryApp"
USER_ID = "demo_user"

runner = Runner(
    agent=scraping_agent_local,
    app_name=APP_NAME,
    session_service=session_service,  
    memory_service=memory_service,    
)

await run_session(
    runner,
    "What are the latest fashion trends?",
    "demo-research"
)


### Session: demo-research

User > What are the latest fashion trends?
Model: > Trend Summary
* The latest fashion trends in 2024 include sustainable fashion, bold colors, and statement accessories.
* Sustainable fashion is gaining popularity, with consumers opting for eco-friendly clothing and accessories.
* Popular fashion styles for winter 2024 include oversized coats, chunky knitwear, and combat boots.
* Men's wear is expected to feature bold prints, bright colors, and statement accessories.
* Athletic wear is also trending, with consumers opting for comfortable and functional clothing.

Key Evidence
* The global sustainable fashion market is expected to reach $10 billion by 2025.
* A survey found that 75% of consumers prefer sustainable fashion over fast fashion.
* Oversized coats are a staple in winter fashion, with designers incorporating bold colors and patterns.
* The use of statement accessories is a key trend in both men's and women's wear.
* Athletic wear is no longer just

In [21]:
APP_NAME = "AutoMemoryApp"
USER_ID = "demo_user"

runner = Runner(
    agent=scraping_agent_local,
    app_name=APP_NAME,
    session_service=session_service,  
    memory_service=memory_service,    
)

await run_session(
    runner,
    "What are the latest tech trends?",
    "demo-research"
)


### Session: demo-research

User > What are the latest tech trends?
Model: > ### Top Emerging Trends
* **AI Goes Physical:** The convergence of AI and robotics is changing the way businesses operate, with a focus on rebuilding and enhancing workflows.
* **Trust and Security:** As technology innovation and adoption accelerate, trust and security become key priorities, with a focus on navigating the challenges of AI and cybersecurity.

### Consumer Sentiment & Drivers
* **Vibe:** Urgent and focused on real business value, with a sense of urgency behind the adoption of new technologies.
* **The "Why":** The pace of change itself has accelerated, driving the need for businesses to move from experimentation to impact, and to prioritize trust and security.

### Key Demographic
* **Tech Leaders:** The trend is driven by technology leaders who are looking to move from experimentation to impact, and to prioritize trust and security.

### Verified Sources
* https://www.deloitte.com/us/en/insigh

In [11]:
APP_NAME = "AutoMemoryApp"
USER_ID = "demo_user"

runner = Runner(
    agent=scraping_agent_local,
    app_name=APP_NAME,
    session_service=session_service,  
    memory_service=memory_service,    
)

await run_session(
    runner,
    "What are the latest AI trends?",
    "demo-research"
)


### Session: demo-research

User > What are the latest AI trends?
Model: > Trend Summary
* The latest AI trends in 2024 include generative AI, Explainable AI (XAI), and Responsible AI.
* Generative AI is gaining popularity, with applications in art, music, and text generation.
* Explainable AI (XAI) is becoming increasingly important, as businesses look to improve transparency and accountability.
* Responsible AI is a key focus, with a focus on fairness, accuracy, and security.
* AI applications for business are expected to grow, with a focus on automation, customer service, and predictive analytics.

Key Evidence
* A survey found that 80% of businesses believe AI will be critical to their success by 2025.
* Generative AI has been used to create realistic artwork and music, and is being explored for use in advertising and marketing.
* XAI is being used to improve decision-making in the financial industry, with applications in risk assessment and portfolio management.
* Responsible AI 

# VIDEO AGENT

In [19]:
import cv2
import ollama

def analyze_video_locally(video_path: str) -> str:
    """
    Analyzes video with explicit error checking for Kaggle/Cloud environments.
    """
    print(f"🎥 Local Analysis: Processing '{video_path}'...")
    
    # 1. Path Check
    if not os.path.exists(video_path):
        return f" Error: Video file not found at {video_path}"

    try:
        # 2. Open Video
        video = cv2.VideoCapture(video_path)
        
        # CRITICAL: Check if OpenCV opened it successfully
        if not video.isOpened():
            return " Error: OpenCV could not open the file. It might be corrupt or the codec is missing."

        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = video.get(cv2.CAP_PROP_FPS)
        
        # Prevent division by zero if FPS is missing
        if fps <= 0:
            return " Error: Could not determine video FPS. Video metadata might be missing."

        duration = total_frames / fps
        print(f"⏳ Video loaded! Duration: {duration:.1f}s")
        
        # 3. Extract Frames
        indices = [int(total_frames * 0.2), int(total_frames * 0.5), int(total_frames * 0.8)]
        descriptions = []
        
        for i, frame_idx in enumerate(indices):
            video.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            success, frame = video.read()
            if not success: 
                print(f"⚠️ Failed to read frame {i+1}")
                continue

            # Encode for Ollama
            _, buffer = cv2.imencode('.jpg', frame)
            image_bytes = buffer.tobytes()
            
            print(f"moondream is looking at Frame {i+1}...")
            response = ollama.chat(
                model='moondream',
                messages=[{
                    'role': 'user',
                    'content': 'Describe this image in detail.',
                    'images': [image_bytes]
                }]
            )
            desc = response['message']['content']
            descriptions.append(f"**Timestamp {frame_idx/fps:.1f}s:** {desc}")
            
        video.release()
        
        if not descriptions:
            return " Error: Video opened, but no frames could be read."

        return (
            f"✅ Local Video Analysis:\n"
            f"⏱️ **Duration:** {duration:.1f}s\n"
            f"🎞️ **Visual Narrative:**\n" + "\n\n".join(descriptions)
        )

    except Exception as e:
        return f" Python Error: {e}"


video_agent_local = LlmAgent(
    model=llm_model,
    name="video_agent_local",
    # instruction="""
    # You are the Video Forensics Engineer.
    # Your tool runs LOCALLY on the machine.
    
    # Task:
    # 1. Run `analyze_video_locally`.
    # 2. Read the text descriptions returned by the tool.
    # 3. Synthesize the findings into a "Vibe Check" report.
    # """,

    instruction="""
        You are the Video Forensics Engineer.
        Your tool runs LOCALLY on the machine.
        
        Task:
        1. Run `analyze_video_locally`.
        2. Read the text descriptions returned by the tool.
        3. Synthesize the findings into a "Vibe Check" report.

        Rules:
        - Always call `analyze_video_locally` exactly once.
        - After the tool returns, produce ONE final Markdown report.
        - Do not output partial tool-call text, placeholders, or phrases like "pending".

        Output format (Markdown):
        - Video Summary (2-5 bullets)
        - Key Visual Evidence (bullets)
        - Notable Text/Logos Seen (if any)
    """,
    tools=[analyze_video_locally]
)

print("✅ Local Video Agent (Ollama Powered) is ready.")


✅ Local Video Agent (Ollama Powered) is ready.


In [ ]:
# import ollama

# try:
#     print("📡 Testing connection to local Ollama...")
#     # Quick test with a text prompt
#     response = ollama.chat(model='moondream:latest', messages=[{'role': 'user', 'content': 'Hello'}])
#     print(f"✅ Success! Ollama replied: {response['message']['content']}")
# except Exception as e:
#     print(f"❌ Connection failed: {e}")
#     print("👉 Make sure the Ollama app is running.")

📡 Testing connection to local Ollama...
✅ Success! Ollama replied: 


In [ ]:
# import ollama
# import numpy as np
# import cv2

# try:
#     print("📡 Testing connection to local Moondream with an image...")
    
#     # 1. Create a dummy black image (just to wake the vision model up)
#     dummy_img = np.zeros((100, 100, 3), dtype=np.uint8)
#     _, buffer = cv2.imencode('.jpg', dummy_img)
#     image_bytes = buffer.tobytes()

#     # 2. Send the image + text prompt
#     response = ollama.chat(
#         model='moondream:latest', 
#         messages=[{
#             'role': 'user', 
#             'content': 'What is in this image? Reply with one word.',
#             'images': [image_bytes] # <--- Crucial for Moondream
#         }]
#     )
#     print(f"✅ Success! Moondream replied: {response['message']['content']}")
# except Exception as e:
#     print(f"❌ Connection failed: {e}")

📡 Testing connection to local Moondream with an image...
✅ Success! Moondream replied: !!!IMAGE NOT GENERATED BY LIGHTENING!!!!!


In [17]:
APP_NAME = "AutoMemoryApp"
USER_ID = "demo_user"

runner_video = Runner(
    agent=video_agent_local,
    app_name=APP_NAME,
    session_service=session_service,  
    memory_service=memory_service,    
)

await run_session(
    runner_video,
    "Watch the video at 'C:/Kaggle-agent/test.mp4'. Describe the visual narrative and the vibe.",
    "demo-video-analysis"
)


### Session: demo-video-analysis

User > Watch the video at 'C:/Kaggle-agent/test.mp4'. Describe the visual narrative and the vibe.
🎥 Local Analysis: Processing 'C:/Kaggle-agent/test.mp4'...
⏳ Video loaded! Duration: 15.0s
moondream is looking at Frame 1...
moondream is looking at Frame 2...
moondream is looking at Frame 3...
Model: > Visual Narrative:

The video starts by showing a painting of a cityscape with buildings and cars. The perspective is from below, looking up at the artwork. Two books are placed in front of the painting.

Then, the video cuts to a billboard with the text "For Bigger Joyrides" written in blue and white letters. The billboard is positioned on top of a building and is angled slightly to the left.

Lastly, the video shows a white rectangular background with black text that reads "For $35. For everyone." The text is centered and stands out against the stark contrast of the white background.

Vibe:

The vibe of the video is playful and attention-grabbing. The u

In [20]:
APP_NAME = "AutoMemoryApp"
USER_ID = "demo_user"

runner_video = Runner(
    agent=video_agent_local,
    app_name=APP_NAME,
    session_service=session_service,  
    memory_service=memory_service,    
)

await run_session(
    runner_video,
    "Watch the video at 'C:/Kaggle-agent/test.mp4'. Describe the visual narrative and the vibe.",
    "demo-video-analysis"
)


### Session: demo-video-analysis

User > Watch the video at 'C:/Kaggle-agent/test.mp4'. Describe the visual narrative and the vibe.
🎥 Local Analysis: Processing 'C:/Kaggle-agent/test.mp4'...
⏳ Video loaded! Duration: 15.0s
moondream is looking at Frame 1...
moondream is looking at Frame 2...
moondream is looking at Frame 3...
Model: > Visual Narrative:

The video starts by showing a painting of a cityscape with buildings and cars. The perspective is from below, looking up at the artwork. Two books are placed in front of the painting.

Then, the video cuts to a billboard with the text "For Bigger Joyrides" written in blue and white letters. The billboard is positioned on top of a building and is angled slightly to the left.

Lastly, the video shows a white rectangular background with black text that reads "For $35. For everyone." The text is centered and stands out against the stark contrast of the white background.

Vibe:

The vibe of the video is playful and attention-grabbing. The u

In [14]:
APP_NAME = "AutoMemoryApp"
USER_ID = "demo_user"

runner_video = Runner(
    agent=video_agent_local,
    app_name=APP_NAME,
    session_service=session_service,  
    memory_service=memory_service,    
)

await run_session(
    runner_video,
    "Watch the video at 'test.mp4'. Describe the visual narrative and the vibe.",
    "demo-video-analysis"
)


### Session: demo-video-analysis

User > Watch the video at 'test.mp4'. Describe the visual narrative and the vibe.
🎥 Local Analysis: Processing 'test.mp4'...
⏳ Video loaded! Duration: 15.0s
moondream is looking at Frame 1...
moondream is looking at Frame 2...
moondream is looking at Frame 3...
Model: > The video 'test.mp4' presents a visual narrative that appears to focus on themes of urban life, technology, and accessibility. The initial scene showcases a painting of a cityscape, suggesting an artistic interpretation of modern urban environments. The inclusion of books on the floor in front of the painting may imply a connection to knowledge or learning within this context.

As the video progresses, it transitions to a billboard with the slogan "For Bigger Joyrides," which could be interpreted as an advertisement for a service or product aimed at enhancing experiences, possibly related to transportation or leisure activities. The depiction of city street scenes below the billboard r